In [18]:
pip install bioservices

  Created wheel for bioservices: filename=bioservices-1.9.0-py3-none-any.whl size=230231 sha256=d3e7449d81c9acb4b645949055e6ab7a165eb7f41acb5a92db04892ff5c33dc3
  Stored in directory: c:\users\reagon.karki\appdata\local\pip\cache\wheels\9b\68\e9\b768a42d74673aa595957798191f0baec83d456f7e18a5a350
  Created wheel for easydev: filename=easydev-0.12.0-py3-none-any.whl size=64231 sha256=f0b543c09bd63ca04ebc402c7cc2c6c48f428a047f93bc44120ee9081acb9fd5
  Stored in directory: c:\users\reagon.karki\appdata\local\pip\cache\wheels\b6\ce\17\2e5a20cb830fe7f5f86da83fc81f7b68dd6e798fae7423cce9
  Created wheel for zope.interface: filename=zope.interface-5.4.0-cp310-cp310-win_amd64.whl size=197622 sha256=cfdf0b0445e467d8e16e59da7d61ff821fd9beadf0aed2e9abc2b20492c85685
  Stored in directory: c:\users\reagon.karki\appdata\local\pip\cache\wheels\21\a9\8b\0bfc5594d8e109d5b25d6b69e0cff14d09d93e3522dcb16d2b
Successfully built bioservices easydev zope.interface
Note: you may need to restart the kernel to use 

In [136]:
import uniprot
import pprint
import io
import pandas as pd

In [137]:
seqids = "NP_000508.1  NP_001018081.3".split()

pairs = uniprot.batch_uniprot_id_mapping_pairs(
  'P_REFSEQ_AC', 'ACC', seqids)

pprint.pprint(pairs, indent=2)

Fetching 2 (P_REFSEQ_AC->ACC) mappings from http://uniprot.org...
[]


In [138]:
ids = ['P37840','P01375']
ids[0]


'P37840'

In [15]:
uniprot_seqids = 'A0QSU3 D9QCH6 A0QL36 P01375'.split()
uniprot_seqids

['A0QSU3', 'D9QCH6', 'A0QL36', 'P01375']

In [16]:
uniprot_data = uniprot.batch_uniprot_metadata(uniprot_seqids[0],'cache')
#pprint.pprint(mapping, indent=2)

Fetching metadata for 6 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata


In [6]:
uniprot_data

{}

In [9]:
for l in open('cache/metadata.0.txt'):
  print(l)

<!DOCTYPE html SYSTEM "about:legacy-compat">

<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>Retrieve/ID mapping</title><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="/" rel="home"/><link href="https://creativecommons.org/licenses/by/4.0/" rel="license"/><link type="image/vnd.microsoft.icon" href="/favicon.ico" rel="shortcut icon"/><link href="/uniprot.min.css2022_02" type="text/css" rel="stylesheet"/><link href="/tippy.css" type="text/css" rel="stylesheet"/><script type="text/javascript">

			var BASE = '/';

		</script><script src="/js-compr.js2022_02" type="text/javascript"></script><script type="text/javascript">

				uniprot.isInternal = false;

				uniprot.namespace = 'uniprot';

				uniprot.releasedate = '2022_02';

			</script><script type="text/javascript">

			;

		</script><meta content="n

In [139]:
from bioservices import UniProt

In [142]:
u = UniProt(verbose=True)
res = u.mapping("P_REFSEQ_AC", "ACC", query='XP_021413590')


INFO    [bioservices.UniProt:363]:  Initialising UniProt service (REST)


In [143]:
res = u.search('P01375', frmt="tab", limit=3, columns="id, entry name, genes, length", database='uniprot')
res

'Entry\tEntry name\tGene names\tLength\nP01375\tTNFA_HUMAN\tTNF TNFA TNFSF2\t233\nQ9BXK5\tB2L13_HUMAN\tBCL2L13 MIL1 CD003\t485\nQ5JX71\tF209A_HUMAN\tFAM209A C20orf106\t171\n'

In [277]:
res = u.retrieve('P01375', frmt='xml', database='uniprot')
res

In [259]:
type(res)

bioservices.xmltools.easyXML

In [74]:
prtns = 'P05067 P37840 Q9BYF1'.split()
prtns

['P05067', 'P37840', 'Q9BYF1']

In [223]:
import urllib
from urllib.parse import urlparse

def ExtractFromUniProt(uniprot_id):

    Uniprot_Dict = []
    # Make a link to the UniProt webservice
    service = UniProt()
    
    for id in uniprot_id:

        #create URL for each uniprot id
        url = 'https://www.uniprot.org/uniprot/'+id+'.txt'   
        print(url)

        #Retrieve data for id in text format
        ret_uprot = urllib.request.urlopen(url)

        print(id)
        id_copy = id
        i=0
        j=0
        id = {}
        id['Disease'] = {}
        id['Reactome'] = {}
        id['Function'] = {}
        id['BioProcess'] = {}
        #print(id)

        #parse each line looking for info about disease, pathway, funcn, bp and so on
        for line in ret_uprot:

            line = line.decode('utf-8')

            #parse lines with disease and extract disease names and omim ids
            if '-!- DISEASE:' in line:
                if ('[MIM:' in line):
                    dis = line.split(':')
                    #dis returns list of splitted text, [1] = name of dis, [2] = OMIM ID, extra chars need cleaning
                    #print(dis[1][1:-5])
                    #print(dis[2][:-1])
                    id['Disease'].update({dis[1][1:-5]:dis[2][:-1]})

            #extract reactome ids and names
            if 'Reactome;' in line:
                ract = line.split(';')
                #ract returns list with reactome id and name, needs cleaning
                id['Reactome'].update({ract[2][1:-2]:ract[1][1:]})
                #print(ract[1][1:])
                #print(ract[2][1:-2])

            #look for functions    
            if ' F:' in line:
                if j<5:
                #take only first 5 entries for now 
                    #print(j)
                    fn = line.split(';')
                    #fn returns list with GO ids and names 
                    id['Function'].update({fn[2][3:]:fn[1][1:]})
                    #print(fn[1][1:])
                    #print(fn[2][3:])
                    j+=1

            #look for biological processes
            if ' P:' in line:
                if i<5:
                #take only first 5 entries for now    
                    #print(i)
                    bp = line.split(';')
                    #bp returns list with GO ids and names 
                    id['BioProcess'].update({bp[2][3:]:bp[1][1:]})
                    #print(bp[1][1:])
                    #print(bp[2][3:])
                    i+=1


        #fetch info about gene, len, seq, mass from api directly
        # Make a query string
        query = "accession:" + str(id_copy)

        # Define a list of columns we want to retrive
        #columnlist = "id,entry name,length,mass,go(biological process),go(molecular function), pathway,feature(TOPOLOGICAL DOMAIN),comment(DISEASE),pathway"
        columnlist = "id,entry name,genes(PREFERRED),length,mass,sequence"

        # Run the remote search
        result = service.search(query, frmt="tab", columns=columnlist)

        df_result = pd.read_table(io.StringIO(result))

        geneName = {'Gene':df_result['Gene names  (primary )'][0]}
        seqLen = {'Length':df_result['Length'][0]}
        #remove comma from mass values
        mass = {'Mass':int(df_result['Mass'][0].replace(",", ""))}
        seq = {'Sequence':df_result['Sequence'][0]}

        id.update(geneName)
        id.update(seqLen)
        id.update(mass)
        id.update(seq)

        Uniprot_Dict.append(id)

    Uniprot_Dict = dict(zip(uniprot_id,Uniprot_Dict))
    
    return(Uniprot_Dict)

In [222]:
test = ExtractFromUniProt(prtns)

https://www.uniprot.org/uniprot/P05067.txt
P05067
https://www.uniprot.org/uniprot/P37840.txt
P37840
https://www.uniprot.org/uniprot/Q9BYF1.txt
Q9BYF1


In [220]:
test

{'P05067': {'Disease': {'Alzheimer disease 1 (AD1)': '104300'},
  'Reactome': {'Platelet degranulation': 'R-HSA-114608',
   'ECM proteoglycans': 'R-HSA-3000178',
   'Regulation of Insulin-like Growth Factor (IGF) transport and uptake by Insulin-like Growth Factor Binding Proteins (IGFBPs)': 'R-HSA-381426',
   'G alpha (q) signalling events': 'R-HSA-416476',
   'G alpha (i) signalling events': 'R-HSA-418594',
   'Lysosome Vesicle Biogenesis': 'R-HSA-432720',
   'Formyl peptide receptors bind formyl peptides and many other ligands': 'R-HSA-444473',
   'TAK1 activates NFkB by phosphorylation and activation of IKKs complex': 'R-HSA-445989',
   'The NLRP3 inflammasome': 'R-HSA-844456',
   'Advanced glycosylation endproduct receptor signaling': 'R-HSA-879415',
   "Deregulated CDK5 triggers multiple neurodegenerative pathways in Alzheimer's disease models": 'R-HSA-8862803',
   'Post-translational protein phosphorylation': 'R-HSA-8957275',
   'TRAF6 mediated NF-kB activation': 'R-HSA-933542',


In [209]:
Uniprot_Dict = dict(zip(prtns,Uniprot_Dict))
#Uniprot_Dict = {k: v for k, v in Uniprot_Dict.items() if v}
#named_drugIndList = {k: v for k, v in named_drugIndList.items() if v}

In [210]:
Uniprot_Dict

{'P05067': {'Disease': {'Alzheimer disease 1 (AD1)': '104300'},
  'Reactome': {'Platelet degranulation': 'R-HSA-114608',
   'ECM proteoglycans': 'R-HSA-3000178',
   'Regulation of Insulin-like Growth Factor (IGF) transport and uptake by Insulin-like Growth Factor Binding Proteins (IGFBPs)': 'R-HSA-381426',
   'G alpha (q) signalling events': 'R-HSA-416476',
   'G alpha (i) signalling events': 'R-HSA-418594',
   'Lysosome Vesicle Biogenesis': 'R-HSA-432720',
   'Formyl peptide receptors bind formyl peptides and many other ligands': 'R-HSA-444473',
   'TAK1 activates NFkB by phosphorylation and activation of IKKs complex': 'R-HSA-445989',
   'The NLRP3 inflammasome': 'R-HSA-844456',
   'Advanced glycosylation endproduct receptor signaling': 'R-HSA-879415',
   "Deregulated CDK5 triggers multiple neurodegenerative pathways in Alzheimer's disease models": 'R-HSA-8862803',
   'Post-translational protein phosphorylation': 'R-HSA-8957275',
   'TRAF6 mediated NF-kB activation': 'R-HSA-933542',


'GO:0004180'

In [62]:
c =list(test['BioProcess'].keys())

In [69]:
test['BioProcess'][c[1]]

'GO:0008344'

In [248]:
#pd.read_xml(res)
pd.read_xml(io.StringIO(res))

TypeError: initial_value must be str or None, not easyXML

In [1]:


# Make a link to the UniProt webservice
service = UniProt()

# Build a query string
query = "Q9AJE3"

# Send the query to UniProt, and catch the search result in a variable
result = service.search(query)

# Inspect the result
print(result)



NameError: name 'UniProt' is not defined

In [342]:


# Make a new query string, and run a remote search at UniProt
new_query = "accession:P37840"
new_result = service.search(new_query)

# Inspect the result
print(new_result)



Entry	Entry name	Status	Protein names	Gene names	Organism	Length
P37840	SYUA_HUMAN	reviewed	Alpha-synuclein (Non-A beta component of AD amyloid) (Non-A4 component of amyloid precursor) (NACP)	SNCA NACP PARK1	Homo sapiens (Human)	140



In [77]:
_valid_columns = [
    # Names & Taxonomy
    'id', 'entry name', 'genes', 'genes(PREFERRED)', 'genes(ALTERNATIVE)',
    'genes(OLN)', 'genes(ORF)', 'organism', 'organism-id', 'protein names',
    'proteome', 'lineage(ALL)', 'lineage-id', 'virus hosts',
    # Sequences
    'fragement', 'sequence', 'length', 'mass', 'encodedon',
    'comment(ALTERNATIVE PRODUCTS)', 'comment(ERRONEOUS GENE MODEL PREDICTION)',
    'comment(ERRONEOUS INITIATION)', 'comment(ERRONEOUS TERMINATION)',
    'comment(ERRONEOUS TRANSLATION)', 'comment(FRAMESHIFT)',
    'comment(MASS SPECTROMETRY)', 'comment(POLYMORPHISM)',
    'comment(RNA EDITING)', 'comment(SEQUENCE CAUTION)',
    'feature(ALTERNATIVE SEQUENCE)', 'feature(NATURAL VARIANT)',
    'feature(NON ADJACENT RESIDUES)',
    'feature(NON STANDARD RESIDUE)', 'feature(NON TERMINAL RESIDUE)',
    'feature(SEQUENCE CONFLICT)', 'feature(SEQUENCE UNCERTAINTY)',
    'version(sequence)',
    # Family and Domains
    'domains', 'domain', 'comment(DOMAIN)', 'comment(SIMILARITY)',
    'feature(COILED COIL)', 'feature(COMPOSITIONAL BIAS)',
    'feature(DOMAIN EXTENT)', 'feature(MOTIF)', 'feature(REGION)',
    'feature(REPEAT)', 'feature(ZINC FINGER)',
    # Function
    'ec', 'comment(ABSORPTION)', 'comment(CATALYTIC ACTIVITY)',
    'comment(COFACTOR)', 'comment(ENZYME REGULATION)', 'comment(FUNCTION)',
    'comment(KINETICS)', 'comment(PATHWAY)', 'comment(REDOX POTENTIAL)',
    'comment(TEMPERATURE DEPENDENCE)', 'comment(PH DEPENDENCE)',
    'feature(ACTIVE SITE)', 'feature(BINDING SITE)', 'feature(DNA BINDING)',
    'feature(METAL BINDING)', 'feature(NP BIND)', 'feature(SITE)',
    # Gene Ontologys
    'go', 'go(biological process)', 'go(molecular function)',
    'go(cellular component)', 'go-id',
    # InterPro
    'interpro',
    # Interaction
    'interactor', 'comment(SUBUNIT)',
    # Publications
    'citation', 'citationmapping',
    # Date of
    'created', 'last-modified', 'sequence-modified', 'version(entry)',
    # Structure
    '3d', 'feature(BETA STRAND)', 'feature(HELIX)', 'feature(TURN)',
    # Subcellular location
    'comment(SUBCELLULAR LOCATION)', 'feature(INTRAMEMBRANE)',
    'feature(TOPOLOGICAL DOMAIN)',
    'feature(TRANSMEMBRANE)',
    # Miscellaneous
    'annotation score', 'score', 'features', 'comment(CAUTION)',
    'comment(TISSUE SPECIFICITY)',
    'comment(GENERAL)', 'keywords', 'context', 'existence', 'tools',
    'reviewed', 'feature', 'families', 'subcellular locations', 'taxonomy',
    'version', 'clusters', 'comments', 'database', 'keyword-id', 'pathway',
    'score',
    # Pathology & Biotech
    'comment(ALLERGEN)', 'comment(BIOTECHNOLOGY)', 'comment(DISRUPTION PHENOTYPE)',
    'comment(DISEASE)', 'comment(PHARMACEUTICAL)', 'comment(TOXIC DOSE)',
    # PTM / Processsing
    'comment(PTM)', 'feature(CHAIN)', 'feature(CROSS LINK)', 'feature(DISULFIDE BOND)',
    'feature(GLYCOSYLATION)', 'feature(INITIATOR METHIONINE)', 'feature(LIPIDATION)',
    'feature(MODIFIED RESIDUE)', 'feature(PEPTIDE)', 'feature(PROPEPTIDE)',
    'feature(SIGNAL)', 'feature(TRANSIT)',
    # Taxonomic lineage
    'lineage(all)', 'lineage(SUPERKINGDOM)', 'lineage(KINGDOM)', 'lineage(SUBKINGDOM)',
    'lineage(SUPERPHYLUM)', 'lineage(PHYLUM)', 'lineage(SUBPHYLUM)', 'lineage(SUPERCLASS)',
    'lineage(CLASS)', 'lineage(SUBCLASS)', 'lineage(INFRACLASS)', 'lineage(SUPERORDER)',
    'lineage(ORDER)', 'lineage(SUBORDER)', 'lineage(INFRAORDER)', 'lineage(PARVORDER)',
    'lineage(SUPERFAMILY)', 'lineage(FAMILY)', 'lineage(SUBFAMILY)', 'lineage(TRIBE)',
    'lineage(SUBTRIBE)', 'lineage(GENUS)', 'lineage(SUBGENUS)', 'lineage(SPECIES GROUP)',
    'lineage(SPECIES SUBGROUP)', 'lineage(SPECIES)', 'lineage(SUBSPECIES)', 'lineage(VARIETAS)',
    'lineage(FORMA)',
    # Taxonomic identifier
    'lineage-id(all)', 'lineage-id(SUPERKINGDOM)', 'lineage-id(KINGDOM)', 'lineage-id(SUBKINGDOM)',
    'lineage-id(SUPERPHYLUM)', 'lineage-id(PHYLUM)', 'lineage-id(SUBPHYLUM)', 'lineage-id(SUPERCLASS)',
    'lineage-id(CLASS)', 'lineage-id(SUBCLASS)', 'lineage-id(INFRACLASS)', 'lineage-id(SUPERORDER)',
    'lineage-id(ORDER)', 'lineage-id(SUBORDER)', 'lineage-id(INFRAORDER)', 'lineage-id(PARVORDER)',
    'lineage-id(SUPERFAMILY)', 'lineage-id(FAMILY)', 'lineage-id(SUBFAMILY)', 'lineage-id(TRIBE)',
    'lineage-id(SUBTRIBE)', 'lineage-id(GENUS)', 'lineage-id(SUBGENUS)', 'lineage-id(SPECIES GROUP)',
    'lineage-id(SPECIES SUBGROUP)', 'lineage-id(SPECIES)', 'lineage-id(SUBSPECIES)', 'lineage-id(VARIETAS)',
    'lineage-id(FORMA)',
    # Cross-references
    'database(db_abbrev)', 'database(EMBL)']

In [145]:
col_list = ['id','genes','sequence','pathway']
new_query = "accession:P01375"
# Run the remote search
result = service.search(new_query, frmt="tab", columns=col_list)

# View the result
print(result)

AttributeError: 'list' object has no attribute 'startswith'

In [87]:
print(service._valid_columns)

['id', 'entry name', 'genes', 'genes(PREFERRED)', 'genes(ALTERNATIVE)', 'genes(OLN)', 'genes(ORF)', 'organism', 'organism-id', 'protein names', 'proteome', 'lineage(ALL)', 'lineage-id', 'virus hosts', 'fragment', 'sequence', 'length', 'mass', 'encodedon', 'comment(ALTERNATIVE PRODUCTS)', 'comment(ERRONEOUS GENE MODEL PREDICTION)', 'comment(ERRONEOUS INITIATION)', 'comment(ERRONEOUS TERMINATION)', 'comment(ERRONEOUS TRANSLATION)', 'comment(FRAMESHIFT)', 'comment(MASS SPECTROMETRY)', 'comment(POLYMORPHISM)', 'comment(RNA EDITING)', 'comment(SEQUENCE CAUTION)', 'feature(ALTERNATIVE SEQUENCE)', 'feature(NATURAL VARIANT)', 'feature(NON ADJACENT RESIDUES)', 'feature(NON STANDARD RESIDUE)', 'feature(NON TERMINAL RESIDUE)', 'feature(SEQUENCE CONFLICT)', 'feature(SEQUENCE UNCERTAINTY)', 'version(sequence)', 'domains', 'domain', 'comment(DOMAIN)', 'comment(SIMILARITY)', 'feature(COILED COIL)', 'feature(COMPOSITIONAL BIAS)', 'feature(DOMAIN EXTENT)', 'feature(MOTIF)', 'feature(REGION)', 'feature(

In [146]:


# Make a query string
query = "accession:P01375"

# Define a list of columns we want to retrive
#columnlist = "id,entry name,length,mass,go(biological process),go(molecular function), pathway,feature(TOPOLOGICAL DOMAIN),comment(DISEASE),pathway"
columnlist = "id,entry name,genes(PREFERRED),length,mass,sequence"

# Run the remote search
result = service.search(query, frmt="tab", columns=columnlist)

# View the result
print(result)



Entry	Entry name	Gene names  (primary )	Length	Mass	Sequence
P01375	TNFA_HUMAN	TNF	233	25,644	MSTESMIRDVELAEEALPKKTGGPQGSRRCLFLSLFSFLIVAGATTLFCLLHFGVIGPQREEFPRDLSLISPLAQAVRSSSRTPSDKPVAHVVANPQAEGQLQWLNRRANALLANGVELRDNQLVVPSEGLYLIYSQVLFKGQGCPSTHVLLTHTISRIAVSYQTKVNLLSAIKSPCQRETPEGAEAKPWYEPIYLGGVFQLEKGDRLSAEINRPDYLDFAESGQVYFGIIAL



In [183]:
df = pd.read_table(io.StringIO(result))
df
#bb = list(df['Sequence'])
#bb

,Entry,Entry name,Gene names (primary ),Length,Mass,Sequence
0,P01375,TNFA_HUMAN,TNF,233,"25,644",MSTESMIRDVELAEEALPKKTGGPQGSRRCLFLSLFSFLIVAGATT...


In [328]:
bb = df['Involvement in disease'][0].split(';')
bb[1]

' DISEASE: Parkinson disease 1, autosomal dominant (PARK1) [MIM:168601]: A complex neurodegenerative disorder characterized by bradykinesia, resting tremor, muscular rigidity and postural instability. Additional features are characteristic postural abnormalities, dysautonomia, dystonic cramps, and dementia. The pathology of Parkinson disease involves the loss of dopaminergic neurons in the substantia nigra and the presence of Lewy bodies (intraneuronal accumulations of aggregated proteins), in surviving neurons in various areas of the brain. The disease is progressive and usually manifests after the age of 50 years, although early-onset cases (before 50 years) are known. The majority of the cases are sporadic suggesting a multifactorial etiology based on environmental and genetic factors. However, some patients present with a positive family history for the disease. Familial forms of the disease usually begin at earlier ages and are associated with atypical clinical features. {ECO:0000

In [204]:
import re
result = re.search('GO.*',a)
result.group(0)

'GO:0005516]<p>'

In [213]:
import re
matches = re.findall(r'\[(.+?)\]',test[0])
matches

['GO:0005516']

In [214]:
test[0]

'calmodulin binding [GO:0005516]'

In [179]:
test = df['Gene ontology (molecular function)'][0].split('; ')
test[0]
a = '<p>calmodulin binding [GO:0005516]<p>'
a

'<p>calmodulin binding [GO:0005516]<p>'

In [220]:
test[0].split(' [')[1]

'GO:0005516]'